In [1]:
import numpy as np
import pandas as pd
import sqlalchemy
import pyodbc
import urllib
import os


server = '10.43.20.148'
database = 'HimsLoan'
username = os.environ.get('HIMS_USERNAME')
password = os.environ.get('HIMS_PASSWORD')
driver='{ODBC Driver 17 for SQL Server}'

params = urllib.parse.quote_plus('DRIVER='+driver+';SERVER='+server+';PORT=1443;DATABASE='+database+';UID='+username+';PWD='+password)
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)

In [2]:
# Use the query that Ajay gave
query = """ 
    select
    p.ProjectNo,
    p.ProjName,
    p.calendaryear,
    p.FullProjectNumber,
    sf.Description FundingSource,
    ids.IDISNo,
    stt.Description ProjectStatus,
    mn.Description MilestoneName,
    md.DateReceived 
    from
    himsloan.dbo.project p join
    himsloan.dbo.LutProjectStatus stt on p.LutProjectStatusCd=stt.LutProjectStatusCd join
    himsloan.dbo.loan l on p.ProjUniqueID=l.ProjUniqueID join
    himsloan.dbo.loanfund lf on l.LoanID=lf.LoanID join
    himsloan.dbo.FundingInfo fi on lf.FundingInfoID=fi.FundingInfoID join
    himsloan.dbo.LutSourceofFunds sf on fi.LutSourceofFundsCD=sf.LutSourceofFundsCD left join
    himsloan.dbo.AssnProjectIDISNo ids on p.ProjUniqueID=ids.Projuniqueid left join
    himsloan.dbo.MilestoneDate md on p.ProjUniqueID=md.ProjUniqueID join
    himsloan.dbo.lutmilestonename mn on md.LutMilestoneNameCD=mn.LutMilestoneNameCD;
    """

In [3]:
df = pd.read_sql(query, engine)

In [4]:
df.head()

,ProjectNo,ProjName,calendaryear,FullProjectNumber,FundingSource,IDISNo,ProjectStatus,MilestoneName,DateReceived
0,100004,"CURRY, C.W. & RUTHIE MAE",2000,000041,New Funds,4896,Closed,Construction Completion(Contract vs Actual),2000-08-16 00:00:00
1,100104,"ROMERO VELIZ, ISAURA",2000,000029,New Funds,4893,Closed,Construction Completion(Contract vs Actual),2000-11-27 00:00:00
2,100124,"ADKINS, FRANCES",1985,850612,New Funds,None,Closed,Construction Completion(Contract vs Actual),1993-06-30 00:00:00
3,100137,TIERRA DEL SOL,2000,000124,New Funds,5891,Closed,Mayor and City Council Authority Approval,2000-05-24 00:00:00
4,100137,TIERRA DEL SOL,2000,000124,New Funds,5891,Closed,Mayor and City Council Authority Approval,2000-05-24 00:00:00


In [5]:
df.to_parquet('../data/HIMS/sample_master.parquet')

In [6]:
df['DateReceived'] = df.DateReceived.astype('str')
df.to_stata('../data/HIMS/sample_master.dta')

## Sample master

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet('../data/HIMS/sample_master.parquet')

In [3]:
df.head()

,ProjectNo,ProjName,calendaryear,FullProjectNumber,FundingSource,IDISNo,ProjectStatus,MilestoneName,DateReceived
0,100004,"CURRY, C.W. & RUTHIE MAE",2000,000041,New Funds,4896,Closed,Construction Completion(Contract vs Actual),2000-08-16
1,100104,"ROMERO VELIZ, ISAURA",2000,000029,New Funds,4893,Closed,Construction Completion(Contract vs Actual),2000-11-27
2,100124,"ADKINS, FRANCES",1985,850612,New Funds,None,Closed,Construction Completion(Contract vs Actual),1993-06-30
3,100137,TIERRA DEL SOL,2000,000124,New Funds,5891,Closed,Mayor and City Council Authority Approval,2000-05-24
4,100137,TIERRA DEL SOL,2000,000124,New Funds,5891,Closed,Mayor and City Council Authority Approval,2000-05-24


In [4]:
print(f'# unique ProjectNo: {df.ProjectNo.nunique()}')
print(f'# unique FullProjectNumber: {df.FullProjectNumber.nunique()}')
print(f'# unique IDISNo: {df.IDISNo.nunique()}')

# unique ProjectNo: 8004
# unique FullProjectNumber: 8004
# unique IDISNo: 3298


In [5]:
df.dtypes

ProjectNo                     int64
ProjName                     object
calendaryear                  int64
FullProjectNumber            object
FundingSource                object
IDISNo                       object
ProjectStatus                object
MilestoneName                object
DateReceived         datetime64[ns]
dtype: object

In [6]:
df['Date'] = df.DateReceived.dt.to_period('M')

In [7]:
df2 = df[['ProjectNo', 'MilestoneName', 'DateReceived', 'Date']].drop_duplicates()
df2['obs'] = df2.groupby(['ProjectNo', 'Date']).cumcount() + 1

In [12]:
df2 = df2.reset_index()

In [13]:
df2['obs_max'] = df2.groupby(['ProjectNo', 'Date']).max()['obs']

TypeError: incompatible index of inserted column with frame index

In [ ]:
df2.head(20)